<a href="https://colab.research.google.com/github/DK45P3R/RA1FRANK/blob/main/RA1FRANK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
from google.colab import files

# ========================
# Função para ler o arquivo
# ========================
def ler_arquivo(caminho):
    with open(caminho, 'r', encoding='utf-8') as f:
        linhas = f.read().splitlines()

    if not linhas:
        raise ValueError("O arquivo está vazio.")

    try:
        n = int(linhas[0])
    except ValueError:
        raise ValueError("A primeira linha do arquivo deve conter um número inteiro.")

    expressoes = linhas[1:]
    if len(expressoes) != n:
        raise ValueError(f"Número de expressões ({len(expressoes)}) não corresponde ao indicado ({n}).")

    return expressoes

# ========================
# Analisador Léxico
# ========================
TOKENS = {
    '\\neg': 'NOT',
    '\\wedge': 'AND',
    '\\vee': 'OR',
    '\\rightarrow': 'IMPLIES',
    '\\leftrightarrow': 'IFF',
    '(': 'LPAREN',
    ')': 'RPAREN',
    'true': 'TRUE',
    'false': 'FALSE'
}

def eh_proposicao(token):
    return re.fullmatch(r'[0-9][0-9a-z]*', token) is not None

def analisador_lexico(expr):
    tokens = []
    i = 0
    while i < len(expr):
        if expr[i].isspace():
            i += 1
            continue

        for k in sorted(TOKENS, key=lambda x: -len(x)):
            if expr.startswith(k, i):
                tokens.append((TOKENS[k], k))
                i += len(k)
                break
        else:
            match = re.match(r'[0-9][0-9a-z]*', expr[i:])
            if match:
                prop = match.group(0)
                tokens.append(('ID', prop))
                i += len(prop)
            else:
                return None
    return tokens

# ========================
# Parser LL(1)
# ========================
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def consumir(self, tipo):
        if self.pos < len(self.tokens) and self.tokens[self.pos][0] == tipo:
            self.pos += 1
            return True
        return False

    def FORMULA(self):
        if self.pos >= len(self.tokens):
            return False

        tipo, _ = self.tokens[self.pos]

        if tipo in ['TRUE', 'FALSE']:
            return self.consumir(tipo)
        elif tipo == 'ID':
            return self.consumir('ID')
        elif tipo == 'LPAREN':
            self.consumir('LPAREN')
            if self.OPERADORUNARIO():
                if self.FORMULA():
                    return self.consumir('RPAREN')
            elif self.OPERADORBINARIO():
                if self.FORMULA() and self.FORMULA():
                    return self.consumir('RPAREN')
        return False

    def OPERADORUNARIO(self):
        return self.consumir('NOT')

    def OPERADORBINARIO(self):
        return self.consumir('AND') or self.consumir('OR') or \
               self.consumir('IMPLIES') or self.consumir('IFF')

    def parse(self):
        return self.FORMULA() and self.pos == len(self.tokens)

# ========================
# Função principal
# ========================
def main(caminho_arquivo):
    try:
        expressoes = ler_arquivo(caminho_arquivo)
    except Exception as e:
        print(f"Erro ao ler arquivo: {e}")
        return

    for expr in expressoes:
        tokens = analisador_lexico(expr)
        if tokens is None:
            print(f"{expr} → inválida")
            continue
        parser = Parser(tokens)
        print(f"{expr} → {'valida' if parser.parse() else 'inválida'}")

# ========================
# Upload de arquivo no Colab
# ========================
uploaded = files.upload()

for filename in uploaded.keys():
    print(f"Processando arquivo: {filename}")
    main(filename)


Saving arquivo1.txt to arquivo1 (1).txt
Saving arquivo2.txt to arquivo2 (1).txt
Saving arquivo3.txt to arquivo3 (1).txt
Processando arquivo: arquivo1 (1).txt
true → valida
false → valida
1abc → valida
Processando arquivo: arquivo2 (1).txt
(\wedge 1a 2b) → valida
(\neg (\vee 1x 3y)) → valida
false → valida
(\leftrightarrow true false) → valida
invalid_token  → inválida
(\neg (\neg 0z)) → valida
Processando arquivo: arquivo3 (1).txt
true → valida
(\neg false) → valida
(\rightarrow 3a (\wedge 1x 2y)) → valida
(\vee 1a) → inválida
